In [1]:
# importing dependencies
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
import requests
import time
import json
import datetime
from pprint import pprint

# Google developer API key
#from config import google_key

# Census developer API key
#from config import census_key

## Data Load, Prep and Review

In [2]:
# reading csv file
StopOriginal = pd.read_csv('MplsStops_base.csv')
# displaying file header and five rows
StopOriginal.head()

#data catg by precinct   
precinct_one = StopOriginal.loc[StopOriginal['policePrecinct'] == 1 ]
precinct_two = StopOriginal.loc[StopOriginal['policePrecinct'] == 2 ]
precinct_three = StopOriginal.loc[StopOriginal['policePrecinct'] == 3 ]
precinct_four = StopOriginal.loc[StopOriginal['policePrecinct'] == 4 ]
precinct_five = StopOriginal.loc[StopOriginal['policePrecinct'] == 5 ]

StopOriginal.head()

,Unnamed: 0,idNum,date,problem,MDC,citationIssued,personSearch,vehicleSearch,preRace,race,gender,lat,long,policePrecinct,neighborhood
0,6823,17-000003,1/1/17 0:00,suspicious,MDC,NaN,NO,NO,Unknown,Unknown,Unknown,44.966617,-93.246458,1,Cedar Riverside
1,6824,17-000007,1/1/17 0:03,suspicious,MDC,NaN,NO,NO,Unknown,Unknown,Male,44.980450,-93.271340,1,Downtown West
2,6825,17-000073,1/1/17 0:23,traffic,MDC,NaN,NO,NO,Unknown,White,Female,44.948350,-93.275380,5,Whittier
3,6826,17-000092,1/1/17 0:33,suspicious,MDC,NaN,NO,NO,Unknown,East African,Male,44.948360,-93.281350,5,Whittier
4,6827,17-000098,1/1/17 0:37,traffic,MDC,NaN,NO,NO,Unknown,White,Female,44.979078,-93.262076,1,Downtown West


### Total Stops

In [3]:
precinct_df = pd.DataFrame(StopOriginal['policePrecinct'].value_counts())
stops = precinct_df['policePrecinct'].sum()
stops

51920

 ### Count of Stops by Race 

In [4]:
race_count = pd.DataFrame(StopOriginal['race'].value_counts())
race_count['%'] = race_count['race'] / stops * 100
race_count['%'] = race_count['%'].map("% {:.2f}".format)
race_count
#plt.pie(race_count['race'])
#plt.show()

,race,%
Black,15220,% 29.31
White,11703,% 22.54
Unknown,9219,% 17.76
East African,2188,% 4.21
Latino,1858,% 3.58
Native American,1516,% 2.92
Other,1348,% 2.60
Asian,647,% 1.25


### Stops Per Precinct by Race

In [5]:
# count of stops per race per precinct
one_race_count = pd.DataFrame(precinct_one['race'].value_counts())
one_race_count = one_race_count.rename(columns={'race':'one'})

two_race_count = pd.DataFrame(precinct_two['race'].value_counts())
two_race_count = two_race_count.rename(columns={'race':'two'})

three_race_count = pd.DataFrame(precinct_three['race'].value_counts())
three_race_count = three_race_count.rename(columns={'race':'three'})

four_race_count = pd.DataFrame(precinct_four['race'].value_counts())
four_race_count = four_race_count.rename(columns={'race':'four'})

five_race_count = pd.DataFrame(precinct_five['race'].value_counts())
five_race_count = five_race_count.rename(columns={'race':'five'})

#stops per precinct by race
stops_per_precinct_by_race = pd.concat([one_race_count,two_race_count,three_race_count,four_race_count,five_race_count], axis=1, sort=True)
stops_per_precinct_by_race = stops_per_precinct_by_race.reset_index()
stops_per_precinct_by_race = stops_per_precinct_by_race.rename(columns={'index':'race'})
stops_per_precinct_by_race = stops_per_precinct_by_race[['race','one','two','three','four','five']]
stops_per_precinct_by_race['Total'] = stops_per_precinct_by_race.sum(axis=1)

stops_per_precinct_by_race

,race,one,two,three,four,five,Total
0,Asian,65,165,53,177,187,647
1,Black,2176,2226,2059,6183,2576,15220
2,East African,286,364,463,127,948,2188
3,Latino,100,429,543,216,570,1858
4,Native American,192,119,750,108,347,1516
5,Other,165,304,273,228,378,1348
6,Unknown,897,1033,2893,2494,1902,9219
7,White,1368,3304,1311,1193,4527,11703


### Precinct Stops by Gender By Gender

In [6]:
one_sex = pd.DataFrame(precinct_one['gender'].value_counts())
two_sex = pd.DataFrame(precinct_two['gender'].value_counts())
three_sex = pd.DataFrame(precinct_three['gender'].value_counts())
four_sex = pd.DataFrame(precinct_four['gender'].value_counts())
five_sex = pd.DataFrame(precinct_five['gender'].value_counts())

precincts_sex = pd.concat([one_sex,two_sex,three_sex,four_sex,five_sex] , axis=1, sort=False)
precincts_sex.columns = ['One','Two','Three','Four','Five']
precincts_sex['Total'] = precincts_sex.sum(axis=1)
precincts_sex['%'] = precincts_sex['Total'] / precincts_sex['Total'].sum() * 100 
precincts_sex['%'] = precincts_sex['%'].map("% {:.2f}".format)
precincts_sex

,One,Two,Three,Four,Five,Total,%
Male,3664,5181,4816,6468,7002,27131,% 62.17
Female,985,2109,1598,2297,3026,10015,% 22.95
Unknown,593,645,1922,1943,1389,6492,% 14.88


###  Stops by Gender By Gender

In [7]:
stop_by_gender = precincts_sex[['Total','%']]
stop_by_gender

,Total,%
Male,27131,% 62.17
Female,10015,% 22.95
Unknown,6492,% 14.88


# Test work below

### At the moment I have been able to split the date column into date and time, Remove the : char and convert the dtype to int

### Still need to either convert the time to a diffrent format or work on my binning method

In [8]:
#split date column into time and date and remove :
StopOriginal['Date'], StopOriginal['Time'] = StopOriginal['date'].str.split(' ', 1).str
StopOriginal['Time'] = StopOriginal['Time'].apply(lambda x: x.replace(':',''))
StopOriginal['Time'] = StopOriginal['Time'].astype('int')
#StopOriginal.dtypes
StopOriginal

,Unnamed: 0,idNum,date,problem,MDC,citationIssued,personSearch,vehicleSearch,preRace,race,gender,lat,long,policePrecinct,neighborhood,Date,Time
0,6823,17-000003,1/1/17 0:00,suspicious,MDC,NaN,NO,NO,Unknown,Unknown,Unknown,44.966617,-93.246458,1,Cedar Riverside,1/1/17,0
1,6824,17-000007,1/1/17 0:03,suspicious,MDC,NaN,NO,NO,Unknown,Unknown,Male,44.980450,-93.271340,1,Downtown West,1/1/17,3
2,6825,17-000073,1/1/17 0:23,traffic,MDC,NaN,NO,NO,Unknown,White,Female,44.948350,-93.275380,5,Whittier,1/1/17,23
3,6826,17-000092,1/1/17 0:33,suspicious,MDC,NaN,NO,NO,Unknown,East African,Male,44.948360,-93.281350,5,Whittier,1/1/17,33
4,6827,17-000098,1/1/17 0:37,traffic,MDC,NaN,NO,NO,Unknown,White,Female,44.979078,-93.262076,1,Downtown West,1/1/17,37
5,6828,17-000111,1/1/17 0:46,traffic,MDC,NaN,NO,NO,Unknown,East African,Male,44.980535,-93.263627,1,Downtown West,1/1/17,46
6,6829,17-000114,1/1/17 0:48,suspicious,MDC,NaN,NO,NO,Unknown,Black,Male,44.980808,-93.273141,1,Downtown West,1/1/17,48
7,6830,17-000120,1/1/17 0:50,traffic,MDC,NaN,NO,NO,Unknown,Other,Female,44.982093,-93.238155,2,Marcy Holmes,1/1/17,50
8,6831,17-000127,1/1/17 0:57,traffic,MDC,NaN,NO,NO,Unknown,White,Male,44.990319,-93.252042,2,Nicollet Island - East Bank,1/1/17,57
9,6832,17-000139,1/1/17 1:05,traffic,MDC,NaN,NO,NO,Unknown,Black,Male,45.013270,-93.308240,4,Folwell,1/1/17,105


In [9]:
bins = [559,1200,1700,2000,0]
groups = ["Morning","AfterNoon","Evening","Night"]

StopOriginal["Time Segments"] = pd.cut(StopOriginal["Time"], bins, labels=groups)

StopOriginal

ValueError: bins must increase monotonically.

In [ ]:
#testing cell

#precinct one (white)
one_white = StopOriginal.loc[(StopOriginal["race"] == 'White') & (StopOriginal["policePrecinct"] == 1)]
#female Stops
one_white_female = test.loc[test['gender'] == 'Female'].count()
#male Stops
one_male = test.loc[test['gender'] == 'Male'].count()

#one_white.groupby([''])

In [ ]:
print(one_white['race'].value_counts())

In [ ]:
# need to look at this again. 



#stops per precinct 
precinct_df = pd.DataFrame(StopOriginal['policePrecinct'].value_counts())
#precinct_df

#percent of stops per precinct
precinct_percent = pd.DataFrame(precinct_df['policePrecinct'] / stops * 100)
precinct_percent = precinct_percent.rename(columns={'policePrecinct':'Percent of Stops'})
precinct_percent["Percent of Stops"] = precinct_percent["Percent of Stops"].map("% {:.2f}".format)
#precinct_percent

precincts = pd.concat([precinct_df,precinct_percent], axis=1)
precincts = precincts.rename(columns={'policePrecinct':'Precinct'})

#adding race catg's to df
precincts['Black'] = ""
precincts['White'] = ""
precincts['Unknown'] = ""
precincts['East African'] = ""
precincts['Latino'] = ""
precincts['Native American'] = ""
precincts['Other'] = ""
precincts['Asian'] = ""

precincts